[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/anytko/ecospat/blob/main/docs/examples/propagule_pressure_raster.ipynb)

In [ ]:
# Uncomment below to run in Google Collab
# pip install ecospat

## Creating and visualizing a propagule pressure raster 

### Step-by-step

In [ ]:
import ecospat.ecospat as ecospat_full
from ecospat.stand_alone_functions import (
    process_species_historical_range,
    analyze_species_distribution,
    analyze_northward_shift,
    calculate_rate_of_change_first_last,
    merge_category_dataframes,
    prepare_gdf_for_rasterization,
    cat_int_mapping,
    rasterize_multiband_gdf_match,
    rasterize_multiband_gdf_world,
    compute_propagule_pressure_range,
    save_raster_to_downloads_range,
    full_propagule_pressure_pipeline,
)

#### Classify historical range edges

In [ ]:
hist_pipeline = ecospat_full.Map()
hist_range = process_species_historical_range(
    new_map=hist_pipeline, species_name="Populus angustifolia"
)

#### Classify modern range edges.

In [ ]:
classified_modern, classified_historic = analyze_species_distribution(
    "Populus angustifolia", record_limit=1000, continent="north_america"
)

#### Calculate the northward movement

In [ ]:
northward_rate_df = analyze_northward_shift(
    gdf_hist=hist_range,
    gdf_new=classified_modern,
    species_name="Populus angustifolia",
)
northward_rate_df = northward_rate_df[
    northward_rate_df["category"].isin(["leading", "core", "trailing"])
]

northward_rate_df["category"] = northward_rate_df["category"].str.title()

#### Calculate population density change

In [ ]:
change = calculate_rate_of_change_first_last(
    classified_historic, classified_modern, "Populus angustifolia", custom_end_year=2025
)


change = change[change["collapsed_category"].isin(["leading", "core", "trailing"])]
change = change.rename(
    columns={
        "collapsed_category": "Category",
        "rate_of_change_first_last": "Rate of Change",
        "start_time_period": "Start Years",
        "end_time_period": "End Years",
    }
)


change["Category"] = change["Category"].str.title()

#### Merge dataframes and prepare for rasterization

In [ ]:
merged = merge_category_dataframes(northward_rate_df, change)

preped_gdf = prepare_gdf_for_rasterization(classified_modern, merged)

preped_gdf_new = cat_int_mapping(preped_gdf)

preped_gdf_new.head()

#### Rasterization

In [ ]:
value_columns = [
    "density",
    "northward_rate_km_per_year",
    "Rate of Change",
    "category_int",
]
raster_show, transform, show_bounds = rasterize_multiband_gdf_match(
    preped_gdf_new, value_columns
)

In [ ]:
import matplotlib.pyplot as plt

# Plotting one of these bands (northward movement rate)

plt.imshow(raster_show[1], cmap="viridis", origin="upper")
plt.colorbar(label="Pressure")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

#### Construct propagule pressure raster.

In [ ]:
pressure_show = compute_propagule_pressure_range(raster_show)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(pressure_show, cmap="viridis", origin="upper")
plt.colorbar(label="Pressure")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

#### (Optional) Save raster as a .tif

In [ ]:
# raster_download = save_raster_to_downloads_range(pressure_show, show_bounds, "Populus angustifolia")

#### View raster

In [ ]:
persistence_map = ecospat_full.Map()
persistence_map.add_basemap("GBIF.Classic")
persistence_map.add_raster(
    "Populus_angustifolia_persistence_raster.tif",
    colormap="viridis",
    legend=True,
    name="Persistence Raster",
)
# persistence_map.add_layer_control()
persistence_map

### Pipeline to generate persistence raster 

In [ ]:
full_show, full_save, show_bounds, save_bounds, gdf_transform, world_transform = (
    full_propagule_pressure_pipeline(classified_modern, northward_rate_df, change)
)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(full_show, cmap="viridis", origin="upper")
plt.colorbar(label="Pressure")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()